## Azure Machine Learning での実験

学習スクリプトの変更箇所
- データセット (引数で引き渡す)
    - `as_mount()` : ファイルをマウントする
    - `as_download()` : ファイルをダウンロードする
- メトリック記録
    - `run.log(xxx)` で実験のメトリックとして記録

In [1]:
from azureml.core import Workspace, Experiment, Environment, ScriptRunConfig, Dataset
from azureml.widgets import RunDetails

In [2]:
ws = Workspace.from_config()

In [3]:
dataset = Dataset.get_by_name(ws, name='cifar10')

In [4]:
experiment = Experiment(workspace=ws, name='3-aml-run')

### 学習スクリプトの実行設定

In [5]:

config = ScriptRunConfig(source_directory='./code/pytorch-cloud',
                         script='train.py',
                         compute_target='local',
                         arguments=[
                            '--data_path', dataset.as_named_input('input').as_download(),
                            '--learning_rate', 0.003,
                            '--momentum', 0.92])

### Environment の呼び出し

In [6]:
env = Environment.get(ws, "pytorch-env")

In [7]:
config.run_config.environment = env

### 実行

In [8]:
run = experiment.submit(config)
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…